# Supervised ML

The goal of this model is to predict the ridership that occurs within the University of Chicago Lyft Program Area. We will do this by using as features the ridership counts of other Chicago community areas, as well as using weather. The labels are the daily ridership counts within the program area.

We will create the model that functions up until the introduction of the University Lyft program and then look at the difference between the predictions and the actual ridership as a rough estimate of the effect of the program on rideshare usage in the area. We will do this by looking at both the change when the program was introduced, as well as when the program was reduced from 10 rides of up to 15 dollars each, to 7 rides up to 10 dollars. 

In [1]:
# read in packages create spark environment
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
%matplotlib inline

spark = SparkSession.builder.appName('supervised').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','4g')])

#print spark configuration settings
spark.sparkContext.getConf().getAll()

[('spark.stage.maxConsecutiveAttempts', '10'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.submit.pyFiles',
  '/root/.ivy2/jars/com.johnsnowlabs.nlp_spark-nlp_2.12-4.4.0.jar,/root/.ivy2/jars/graphframes_graphframes-0.8.2-spark3.1-s_2.12.jar,/root/.ivy2/jars/com.typesafe_config-1.4.2.jar,/root/.ivy2/jars/org.rocksdb_rocksdbjni-6.29.5.jar,/root/.ivy2/jars/com.amazonaws_aws-java-sdk-bundle-1.11.828.jar,/root/.ivy2/jars/com.github.universal-automata_liblevenshtein-3.0.0.jar,/root/.ivy2/jars/com.google.cloud_google-cloud-storage-2.16.0.jar,/root/.ivy2/jars/com.navigamez_greex-1.0.jar,/root/.ivy2/jars/com.johnsnowlabs.nlp_tensorflow-cpu_2.12-0.4.4.jar,/root/.ivy2/jars/it.unimi.dsi_fastutil-7.0.12.jar,/root/.ivy2/jars/org.projectlombok_lombok-1.16.8.jar,/root/.ivy2/jars/com.google.guava_guava-31.1-jre.jar,/root/.ivy2/jars/com.google.guava_failureaccess-1.0.1.jar,/root/.ivy2/jars/com.google.guava_listenablefuture-9999.0-empty-to-avoid-conflict-

### Reading in cleaned data, partitioning

In [24]:
# read in rideshare data for all years, concatenate, create appropriate partitioning
# we are dropping 2020 because covid will affect the performance of our model

df_2018 = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/rideshare/processed_data/rides_2018.csv", inferSchema=True, header=True)
df_2019 = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/rideshare/processed_data/rides_2019.csv", inferSchema=True, header=True)
df_2021 = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/rideshare/processed_data/rides_2021.csv", inferSchema=True, header=True)
df_2022 = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/rideshare/processed_data/rides_2022.csv", inferSchema=True, header=True)
df_2023 = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/rideshare/processed_data/rides_2023.csv", inferSchema=True, header=True)

# dropping new columns in 2023
df_2023 = df_2023.drop('Shared Trip Match','Percent Time Chicago','Percent Distance Chicago')

df_all = df_2018.union(df_2019).union(df_2021).union(df_2022).union(df_2023)
df_all.show(5)

+--------------------+-------------------+-------------------+-------+-----+------------+-------------+-----------+------------+----+---+-----+-------------+--------------+-------------+--------------+-----+------------+----+---+
|                  ID|    start_timestamp|      end_timestamp|seconds|miles|pickup_tract|dropoff_tract|pickup_area|dropoff_area|Fare|Tip|total|   pickup_lat|    pickup_lon|  dropoff_lat|   dropoff_lon|month|day_of_month|hour|day|
+--------------------+-------------------+-------------------+-------+-----+------------+-------------+-----------+------------+----+---+-----+-------------+--------------+-------------+--------------+-----+------------+----+---+
|625e77ae6e0ff7191...|2018-11-06 19:00:00|2018-11-06 19:15:00|   1142|  5.8| 17031063400|  17031010400|          6|           1|12.5|  0| 15.0|41.9346591566|-87.6467297286| 42.004764559| -87.659122427|   11|           6|  19|  3|
|62945fdb2e70957f0...|2018-11-06 19:00:00|2018-11-06 19:00:00|    341|  1.2| 170

In [5]:
#display number of records by partition
def displaypartitions(df):
    #number of records by partition
    num = df.rdd.getNumPartitions()
    print("Partitions:", num)
    df.withColumn("partitionId", F.spark_partition_id())\
        .groupBy("partitionId")\
        .count()\
        .orderBy(F.asc("count"))\
        .show(num)

df_all.rdd.getNumPartitions()
displaypartitions(df_all)


Partitions: 544


+-----------+------+
|partitionId| count|
+-----------+------+
|         42|305254|
|         41|305316|
|         40|305420|
|         38|305471|
|         39|305480|
|         37|305618|
|         36|305676|
|         35|305871|
|         34|305890|
|         33|305962|
|         32|305971|
|         31|306010|
|         29|306031|
|         30|306038|
|         28|306086|
|         27|306127|
|         26|306402|
|         25|306467|
|         24|306633|
|         23|306731|
|         22|307226|
|        243|328837|
|        242|328975|
|        241|329131|
|        240|329163|
|        239|329209|
|        237|329245|
|        235|329263|
|        238|329263|
|        234|329311|
|        236|329315|
|        232|329332|
|        233|329344|
|        231|329373|
|        228|329389|
|        229|329390|
|        227|329399|
|        226|329410|
|        225|329410|
|        224|329418|
|        230|329427|
|        223|329428|
|        220|329461|
|        222|329481|
|        221|

In [25]:
# repartitioning to 600 partitions, seems to be balanced now. 
df_all = df_all.repartition(600)
#displaypartitions(df_all)

In [26]:
# we will need a year column in this model
df_all = df_all.withColumn('year', F.year(df_all.start_timestamp))

In [8]:
df_all.printSchema()

root
 |-- ID: string (nullable = true)
 |-- start_timestamp: timestamp (nullable = true)
 |-- end_timestamp: timestamp (nullable = true)
 |-- seconds: integer (nullable = true)
 |-- miles: double (nullable = true)
 |-- pickup_tract: long (nullable = true)
 |-- dropoff_tract: long (nullable = true)
 |-- pickup_area: integer (nullable = true)
 |-- dropoff_area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tip: integer (nullable = true)
 |-- total: double (nullable = true)
 |-- pickup_lat: double (nullable = true)
 |-- pickup_lon: double (nullable = true)
 |-- dropoff_lat: double (nullable = true)
 |-- dropoff_lon: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- day_of_month: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- year: integer (nullable = true)



## Next steps

I'm assuming we are predicting using the full dataset and not restricting ourselves to being within the program hours.

I started writing code that goes through the steps that I think will probably be necessary. The code is unfinished because I ran out of time to test in all or formally think through the problems I was seeing. Feel free to change things or make your own assumptions.

Here is the process that I was thinking of. I was trying all this on a sample dataframe so I could code faster.
1. Get Daily counts for each community area
2. pivot so that there is a column for each community area (y is when hyde park or woodlawn or kenwood are 1, otherwise the column is a feature)
3. merge with daily weather data
4. separate out y (counts for every day in program area) and X (column of counts for each community area outside of the program area)
5. filter for pre-program rides.
- Research which model works best and which one is the most parallelizable
6. create supervised model on all that data
7. predict the next month or so of counts after sept 29 2021
8. Graph predictions versus reality
9. maybe do the same thing in 2023 once data is available

In [41]:
# take a sample to test these operations out on first
#sample_df = df_all.sample(fraction=1/1000000)

# get only the columns needed for the model
selected_columns = ["pickup_area","dropoff_area","day","month","year","ID"]
selected_df = df_all.select(selected_columns)

**Daily counts for each community area**

we had to group by pickup area and dropoff area seperately- daily counts of number of trips to that particular community area when it was either a pickup or dropoff area

In [46]:
# Calculate daily counts for pickup areas
pickup_counts = selected_df.groupby('day', 'month', 'year', 'pickup_area').count().withColumnRenamed('count', 'pickup_count')
pickup_counts = pickup_counts.withColumnRenamed('pickup_area', 'area')

In [47]:
# Calculate daily counts for dropoff areas
dropoff_counts = selected_df.groupby('day', 'month', 'year', 'dropoff_area').count().withColumnRenamed('count', 'dropoff_count')
dropoff_counts = dropoff_counts.withColumnRenamed('dropoff_area', 'area')

In [48]:
pickup_counts = pickup_counts.withColumn('dropoff_count', F.lit(0))
dropoff_counts = dropoff_counts.withColumn('pickup_count', F.lit(0))

# ensuring same column order
pickup_counts = pickup_counts.select('day', 'month', 'year', 'area', 'pickup_count', 'dropoff_count')
dropoff_counts = dropoff_counts.select('day', 'month', 'year', 'area', 'pickup_count', 'dropoff_count')

# Union the pickup and dropoff dataframes
combined_df = pickup_counts.union(dropoff_counts)

# Group by day, month, year, and area, summing up the counts
daily_counts_by_area = combined_df.groupby('day', 'month', 'year', 'area').sum('pickup_count', 'dropoff_count')

# the relatively smaller numbers are mostly a result of the sample size, should be fine when we 
# make it to the entire dataframe
daily_counts_by_area = daily_counts_by_area.withColumn('total_counts', F.col('sum(pickup_count)') + F.col('sum(dropoff_count)'))
daily_counts_by_area.drop('sum(pickup_count)','sum(dropoff_count)')
daily_counts_by_area.show(10)

+---+-----+----+----+-----------------+------------------+------------+
|day|month|year|area|sum(pickup_count)|sum(dropoff_count)|total_counts|
+---+-----+----+----+-----------------+------------------+------------+
|  6|   11|2018|  40|             4020|              3638|        7658|
|  2|   11|2018|  67|             2938|              2666|        5604|
|  2|   11|2018|  29|             5383|              5348|       10731|
|  6|   12|2018|  43|             9437|              8841|       18278|
|  7|    2|2019|  28|           119193|            125444|      244637|
|  5|    1|2019|  24|            81650|             75753|      157403|
|  4|    8|2019|  71|             4619|              4354|        8973|
|  6|    5|2019|  25|            15544|             14777|       30321|
|  7|    2|2019|  43|            10001|              9494|       19495|
|  6|    4|2019|  48|             1595|              1655|        3250|
+---+-----+----+----+-----------------+------------------+------

Pivoting the dataset for community areas

In [49]:
#keep datetime as part of the pivot. 

# pivot so that each community area is a column
# one row for each day, each column represents a community area (with its entry being daily count of rides for that area).

# Pivot the DataFrame
pivoted_df = daily_counts_by_area.groupBy("day", "month", "year").pivot("area").sum("total_counts")

# Show the results
pivoted_df.show()

+---+-----+----+-----+-----+-----+-----+-----+------+------+------+----+----+----+----+----+-----+-----+-----+----+----+-----+----+-----+-----+-----+------+-----+----+-----+------+-----+-----+-----+------+-----+-----+-----+----+----+-----+-----+----+-----+-----+-----+-----+----+----+----+----+-----+----+----+----+----+----+---+-----+----+----+----+-----+----+----+----+----+----+-----+----+-----+-----+----+-----+----+----+----+----+-----+-----+
|day|month|year|    1|    2|    3|    4|    5|     6|     7|     8|   9|  10|  11|  12|  13|   14|   15|   16|  17|  18|   19|  20|   21|   22|   23|    24|   25|  26|   27|    28|   29|   30|   31|    32|   33|   34|   35|  36|  37|   38|   39|  40|   41|   42|   43|   44|  45|  46|  47|  48|   49|  50|  51|  52|  53|  54| 55|   56|  57|  58|  59|   60|  61|  62|  63|  64|  65|   66|  67|   68|   69|  70|   71|  72|  73|  74|  75|   76|   77|
+---+-----+----+-----+-----+-----+-----+-----+------+------+------+----+----+----+----+----+-----+-----+

Read in weather data, merge with rideshare data

In [12]:
df_weather_1 = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/weather/chicago 2018-01-01 to 2020-01-01.csv", inferSchema=True, header=True)
df_weather_2 = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/weather/chicago 2020-01-01 to 2022-08-31.csv", inferSchema=True, header=True)
df_weather_3 = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/weather/chicago 2022-09-01 to 2022-12-31.csv", inferSchema=True, header=True)
df_weather_4 = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/weather/chicago 2023-01-01 to 2023-11-22.csv", inferSchema=True, header=True)
df_weather = df_weather_1.union(df_weather_2).union(df_weather_3).union(df_weather_4)

In [50]:
df_weather = df_weather.select('name', 'datetime', 'temp','precip','snow','snowdepth','sunset')
df_weather = df_weather.withColumn('sunset', F.concat(F.hour(df_weather.sunset),F.minute(df_weather.sunset)))
df_weather.printSchema()
pivoted_df.printSchema()
# name, datetime, temp, precip, snow, snowdepth, sunset.
# merge on datetime- keep datetime as part of the pivot. 

root
 |-- name: string (nullable = true)
 |-- datetime: date (nullable = true)
 |-- temp: double (nullable = true)
 |-- precip: double (nullable = true)
 |-- snow: double (nullable = true)
 |-- snowdepth: double (nullable = true)
 |-- sunset: string (nullable = true)

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- 1: long (nullable = true)
 |-- 2: long (nullable = true)
 |-- 3: long (nullable = true)
 |-- 4: long (nullable = true)
 |-- 5: long (nullable = true)
 |-- 6: long (nullable = true)
 |-- 7: long (nullable = true)
 |-- 8: long (nullable = true)
 |-- 9: long (nullable = true)
 |-- 10: long (nullable = true)
 |-- 11: long (nullable = true)
 |-- 12: long (nullable = true)
 |-- 13: long (nullable = true)
 |-- 14: long (nullable = true)
 |-- 15: long (nullable = true)
 |-- 16: long (nullable = true)
 |-- 17: long (nullable = true)
 |-- 18: long (nullable = true)
 |-- 19: long (nullable = true)
 |-- 20: long (n

In [52]:
df_weather.show(2)

+-------+----------+----+------+----+---------+------+
|   name|  datetime|temp|precip|snow|snowdepth|sunset|
+-------+----------+----+------+----+---------+------+
|chicago|2018-01-01|-2.7|   0.0| 0.0|      0.6|  1630|
|chicago|2018-01-02|-0.7|   0.0| 0.0|      0.3|  1631|
+-------+----------+----+------+----+---------+------+
only showing top 2 rows



In [53]:
from pyspark.sql.functions import concat_ws, to_date, lpad

# pad the days and months with zeros for converting to datetime
padded_month = lpad(pivoted_df["month"], 2, "0")
padded_day = lpad(pivoted_df["day"], 2, "0")

# concatenate columns and create datetime column for merging
date_string_column = concat_ws("-", pivoted_df["year"], padded_month, padded_day)
pivoted_df = pivoted_df.withColumn('datetime',F.to_date(date_string_column,'yyy-MM-dd'))
pivoted_df.show(2)

+---+-----+----+-----+----+-----+-----+-----+-----+-----+------+---+----+----+----+----+----+----+-----+----+----+----+----+-----+-----+-----+-----+-----+----+----+-----+----+----+-----+------+-----+----+----+----+---+----+----+----+-----+----+-----+----+----+----+---+----+----+----+----+---+----+---+---+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+----+-----+-----+----------+
|day|month|year|    1|   2|    3|    4|    5|    6|    7|     8|  9|  10|  11|  12|  13|  14|  15|   16|  17|  18|  19|  20|   21|   22|   23|   24|   25|  26|  27|   28|  29|  30|   31|    32|   33|  34|  35|  36| 37|  38|  39|  40|   41|  42|   43|  44|  45|  46| 47|  48|  49|  50|  51| 52|  53| 54| 55|   56|  57|  58|  59|  60|  61|  62|  63|  64|  65|  66|  67|  68|  69|  70|  71|  72|  73| 74|  75|   76|   77|  datetime|
+---+-----+----+-----+----+-----+-----+-----+-----+-----+------+---+----+----+----+----+----+----+-----+----+----+----+----+-----+-----+----

In [58]:
# create the program area sum column
pivoted_df = pivoted_df.withColumn('area_sums', pivoted_df['39'] + pivoted_df['41'] + pivoted_df['42'])
pivoted_df.show(2)

+---+-----+----+-----+----+-----+-----+-----+-----+-----+------+---+----+----+----+----+----+----+-----+----+----+----+----+-----+-----+-----+-----+-----+----+----+-----+----+----+-----+------+-----+----+----+----+---+----+----+----+-----+----+-----+----+----+----+---+----+----+----+----+---+----+---+---+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+----+-----+-----+----------+---------+
|day|month|year|    1|   2|    3|    4|    5|    6|    7|     8|  9|  10|  11|  12|  13|  14|  15|   16|  17|  18|  19|  20|   21|   22|   23|   24|   25|  26|  27|   28|  29|  30|   31|    32|   33|  34|  35|  36| 37|  38|  39|  40|   41|  42|   43|  44|  45|  46| 47|  48|  49|  50|  51| 52|  53| 54| 55|   56|  57|  58|  59|  60|  61|  62|  63|  64|  65|  66|  67|  68|  69|  70|  71|  72|  73| 74|  75|   76|   77|  datetime|area_sums|
+---+-----+----+-----+----+-----+-----+-----+-----+-----+------+---+----+----+----+----+----+----+-----+----+----+----+-

In [59]:
# Drop day and the community areas that were summed earlier, will use year and month in regression
pivoted_df = pivoted_df.drop('day','39','41','42')

df_weather = df_weather.withColumn('datetime', F.to_date('datetime', 'yyyy-MM-dd'))

# Perform the left join with weather data
merged_df = pivoted_df.join(df_weather, on='datetime', how='left')
merged_df.show(2)

+----------+-----+----+-----+----+-----+-----+-----+-----+-----+------+---+----+----+----+----+----+----+-----+----+----+----+----+-----+-----+-----+-----+-----+----+----+-----+----+----+-----+------+-----+----+----+----+---+----+----+-----+----+----+----+---+----+----+----+----+---+----+---+---+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+----+-----+-----+---------+-------+----+------+----+---------+------+
|  datetime|month|year|    1|   2|    3|    4|    5|    6|    7|     8|  9|  10|  11|  12|  13|  14|  15|   16|  17|  18|  19|  20|   21|   22|   23|   24|   25|  26|  27|   28|  29|  30|   31|    32|   33|  34|  35|  36| 37|  38|  40|   43|  44|  45|  46| 47|  48|  49|  50|  51| 52|  53| 54| 55|   56|  57|  58|  59|  60|  61|  62|  63|  64|  65|  66|  67|  68|  69|  70|  71|  72|  73| 74|  75|   76|   77|area_sums|   name|temp|precip|snow|snowdepth|sunset|
+----------+-----+----+-----+----+-----+-----+-----+-----+-----+------+---+-

In [61]:
merged_df = merged_df.drop('datetime','name') #since no longer needed. 
merged_df.show(2)

+-----+----+-----+----+-----+-----+-----+-----+-----+------+---+----+----+----+----+----+----+-----+----+----+----+----+-----+-----+-----+-----+-----+----+----+-----+----+----+-----+------+-----+----+----+----+---+----+----+-----+----+----+----+---+----+----+----+----+---+----+---+---+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+----+-----+-----+---------+----+------+----+---------+------+
|month|year|    1|   2|    3|    4|    5|    6|    7|     8|  9|  10|  11|  12|  13|  14|  15|   16|  17|  18|  19|  20|   21|   22|   23|   24|   25|  26|  27|   28|  29|  30|   31|    32|   33|  34|  35|  36| 37|  38|  40|   43|  44|  45|  46| 47|  48|  49|  50|  51| 52|  53| 54| 55|   56|  57|  58|  59|  60|  61|  62|  63|  64|  65|  66|  67|  68|  69|  70|  71|  72|  73| 74|  75|   76|   77|area_sums|temp|precip|snow|snowdepth|sunset|
+-----+----+-----+----+-----+-----+-----+-----+-----+------+---+----+----+----+----+----+----+-----+----+----+----

In [62]:
# write completed data to a GCS bucket so we don't have to rerun things
merged_df.write.option("header", "true").csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/rideshare/processed_data/supervised_dataset.csv")

# ML Model Psuedo Code

Assumes a dataframe df_merged where the the columns are daily program ride counts followed by daily ride counts for other community areas. Then the daily weather columns with datetime dropped after the merge. 

I folowed fairly closely the ML from one of Ashish's notebooks. I didn't do cross validation yet because I wanted to know how long it takes to run a single regression. 

In [2]:
# skip all the above and just run this line to get the final dataset loaded in to use with the ML model
merged_df = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/rideshare/processed_data/supervised_dataset.csv", inferSchema=True, header=True)

In [25]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer,OneHotEncoder, IndexToString, VectorAssembler
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import FloatType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

In [37]:
# input features are everythin but the area_sums column which is what we are trying to predict
input_features = ['month','year','1','2','3','4','5','6','7','8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20','21','22','23','24','25',
 '26','27','28','29','30','31','32','33','34','35','36','37','38','40','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59','60','61','62','63','64','65','66',
 '67','68','69','70','71','72','73','74','75','76','77','temp','precip','snow','snowdepth','sunset']

# adding the handleInvalid = 'skip' allows this to run- double check what it is doing
vectorAssembler = VectorAssembler(inputCols=input_features,
                                  outputCol="features", handleInvalid='skip')
#df = vectorAssembler.transform(merged_df)
#df.show(5)
#train_df, test_df =  df.randomSplit([0.7, 0.3],0.0)

# splitting first and then doing vector assembly to avoid errors
train_df, test_df = merged_df.randomSplit([.7,.3],seed=1234)
train_df = vectorAssembler.transform(merged_df)
test_df = vectorAssembler.transform(merged_df)

In [38]:
train_df.show(5)

+-----+----+-----+-----+-----+-----+-----+------+------+------+----+----+----+----+----+-----+-----+-----+----+----+-----+----+-----+-----+-----+------+-----+----+----+------+-----+-----+-----+------+-----+-----+-----+----+----+-----+----+-----+-----+----+----+---+----+-----+----+----+---+----+----+---+-----+----+----+----+-----+-----+----+----+----+----+----+----+-----+-----+----+-----+----+----+----+----+-----+-----+---------+----+------+----+---------+------+--------------------+
|month|year|    1|    2|    3|    4|    5|     6|     7|     8|   9|  10|  11|  12|  13|   14|   15|   16|  17|  18|   19|  20|   21|   22|   23|    24|   25|  26|  27|    28|   29|   30|   31|    32|   33|   34|   35|  36|  37|   38|  40|   43|   44|  45|  46| 47|  48|   49|  50|  51| 52|  53|  54| 55|   56|  57|  58|  59|   60|   61|  62|  63|  64|  65|  66|  67|   68|   69|  70|   71|  72|  73|  74|  75|   76|   77|area_sums|temp|precip|snow|snowdepth|sunset|            features|
+-----+----+-----+-----+

In [39]:
# i'm not sure if train_df and test_df is right here, look into this
train_df = train_df.na.drop()  # Remove rows with null values

# Train Model
from pyspark.ml.regression import LinearRegression

#Elastic Net
lr = LinearRegression(featuresCol = 'features', labelCol='area_sums', regParam=0.3, elasticNetParam=0.8, maxIter=10)
lrm = lr.fit(train_df)

#coefficients
print("Coefficients: " + str(lrm.coefficients))
print("Intercept: " + str(lrm.intercept))

#model summary
print("RMSE: %f" % lrm.summary.rootMeanSquaredError)
print("r2: %f" % lrm.summary.r2)

# Run the classifier on the test set
predictions = lrm.transform(test_df)
predictions.select('features','area_sums','prediction').show(5,truncate=False)

Coefficients: [-66.72841290960203,2096.320335854964,0.07737454071173874,0.035134305792045864,0.02137694958199008,0.0543922732894695,0.020625005325430445,0.0022373637819797197,0.0032653293629435007,0.0037847522795157107,0.04699003465926484,0.2534178310079206,0.2393992387806387,0.2968434471731084,0.6882574683561236,0.09583256594707623,0.06473861640623363,0.04318527565506878,0.5697475959137256,0.574714192173466,0.004413737015360228,0.028274619319269237,0.04111630162648094,0.008932993003971074,-0.013839727437326598,0.0013968887010071549,-0.0964834011376087,-0.30392121119556476,-0.038722182947819254,0.02048313918158278,-0.07767640509988266,0.06659548001348663,0.06074265323601359,0.012613139708165286,0.04981127572149861,0.09864269630223645,-0.015143088242105246,-0.1503136291113701,0.33173139285575554,-0.0172261863312499,0.22590390768630747,-0.023426737812655345,0.11547744828368202,0.5731906872508918,-0.08754040754088893,1.8322526823373078,0.03800847680519966,0.173872133558346,2.0806163697074

In [40]:
#print evaluation metrics
e = RegressionEvaluator(labelCol='area_sums', predictionCol= 'prediction', metricName= 'rmse')

# Root Mean Square Error
rmse = e.evaluate(predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = e.evaluate(predictions, {e.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = e.evaluate(predictions, {e.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = e.evaluate(predictions, {e.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 5753.786


MSE: 33106050.484


MAE: 4626.275


r2: 0.907


# pseudocode if we want to do cross validation

In [ ]:
#taken from ashish's notebook- to check if we need to convert anything to labels
def labelForResults(s):
    if s == 'Fail':
        return 0.0
    elif s == 'Pass w/ Conditions' or s == 'Pass':
        return 1.0
    else:
        return -1.0
    
#add labels to the original dataset    
label = UserDefinedFunction(labelForResults, DoubleType())
labeledData = df.select(df.Violations,label(df.Results).alias('label')).where('label >= 0')
labeledData.show(10, truncate=False)

In [45]:
# an open question for me is that in Ashish's code and online examples, the model is fit and tested on the same full dataset. which is confusing to me
# I would think you would fit on training and transform the test set. Something to look into.

# look to adjust the parameters regparam and elastic net

paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.1, 0.01]).addGrid(
    lr.elasticNetParam, [0.2, 0.4,0.6, 0.8,1.0]).build() 
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(),
                          numFolds=10) 

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(train_df)


23/11/25 03:43:39 WARN org.apache.spark.sql.execution.CacheManager: Asked to cache already cached data.
23/11/25 03:43:39 WARN org.apache.spark.sql.execution.CacheManager: Asked to cache already cached data.


IllegalArgumentException: label does not exist. Available: month, year, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, area_sums, temp, precip, snow, snowdepth, sunset, features, CrossValidator_82ced15f4dc7_rand, prediction

In [ ]:

# Save the model in a location so we don't have to rerun this
model_path = "gs://msca-bdp-student-gcs/bdp-rideshare-project/models/supervised_model/"
cvModel.write().save(model_path)

# Read the model back in
cvModelRead = CrossValidatorModel.read().load(model_path)

# Make predictions on test documents. cvModel uses the best model found (lrModel).
predictions = cvModel.transform(test_df)